# Fraunhofer Publica

Dieses Skript führt eine Abfrage in der Fraunhofer Publica durch. Das Ergebnis wird darauf hin geparst, ob Dateianhänge zu diesem Dokument verfügbar sind. Falls ja, werden ein paar Daten extrahiert und in eine Datei geschrieben. Diese Datei kann dann später ausgewertet werden. z. B. bezüglich der Dateiformate.<br>
<table table border="1" align="left">
 <tbody>
    <tr>
      <td>Autor: </td>
      <td style="text-align: left">Antje Schroeder (antje.schroeder@zv.fraunhofer.de)</td>
    </tr>
    <tr>
      <td>Stand: </td>
      <td style="text-align: left">05/19/2023</td>
    </tr>
    <tr>
      <td>Zustand: </td>
      <td style="text-align: left">lauffähig, fertig</td>
    </tr>
 </tbody>
</table><br>
<br><br><br><br><br>
Dieses Skript ermittelt aus der Fraunhofer Publica die Datensätze, an denen eine Datei (Bitstream) angehängt ist. Pro Datensatz können mehrere Dateianhänge angefügt sein, i. d. R. ist eine Datei mit Lizenzinformationen und das Dokument angefügt. Es wird zunächst das Item ermittelt, dann die an einem Item hängenden Bundles, dann die zu einem Bundle gehörenden Bitstreams. Es werden verschiedene Metadaten des Items, des Bundles und des Bitstreams aus den zurückgelieferten Daten ermittelt und in eine Datei geschrieben. <br>

* Ausgabedatei: publica_list_of_publications_with_attached_files.csv</br>
* Datenverzeichnis: ./data</br>

Die Informationen werden iterativ aus den verschiedenen Entitäten der Publica ermittelt: 

* Item-URL: <pre>
  https://publica-rest.fraunhofer.de/server/api/core/items/ [item UUID]
  https://publica-rest.fraunhofer.de/server/api/core/items/1930c37c-7e40-4630-bcb2-98417c5add8d/ </pre>
            
* Bundle-URL: <pre>
  https://publica-rest.fraunhofer.de/server/api/core/items/ [item UUID] /bundles
  https://publica-rest.fraunhofer.de/server/api/core/items/1930c37c-7e40-4630-bcb2-98417c5add8d/bundles </pre>
  
* Bitstream-URL: <pre>
  https://publica-rest.fraunhofer.de/server/api/core/bundles/ [bundles UUID ] /bitstreams
  https://publica-rest.fraunhofer.de/server/api/core/bundles/6830547c-da91-4133-a49d-d42fc6b943e2/bitstreams </pre>

Dieses Skript benutzt die DSpace 7 RESTfull-API:
* https://publica.fraunhofer.de/info/api
* https://wiki.lyrasis.org/display/DSPACE/DSpace+7+REST+API

In [69]:
# Import modules 

import requests
import json
import pandas as pd
import csv

In [70]:
# Define environment variables 

# Define base parameter for url construction
base_url_item = "https://publica-rest.fraunhofer.de/server/api/core/items/"
base_url_download = "https://publica.fraunhofer.de/bitstreams/"

app_bundles = "/bundles"
app_download = "/download"

# count the number of found records
no_of_records = 0

# Define fields for output
fields = ['item_uuid', 'item_handle', 'bundles_uuid', 'bundles_name', 'bitstream_uuid', 'bitstream_name', 'file_format', 'orig_file_name', 'orig_file_format', 'bitstream_bundleName', 'bitstream_size', 'dwl_url']

output_dir = "./data/"
output_file = output_dir + "publica_list_of_publications_with_attached_files.csv"

# Open output file
with open(output_file, 'w', newline='', encoding="utf-8") as f: # prepare output file 
    write = csv.writer(f)
    write.writerow(fields) # write header to outputfile

In [71]:
# Create URl and fetch some initial data to get an overview about the number of records found
url = "https://publica-rest.fraunhofer.de/server/api/discover/search/objects?configuration=defaultConfiguration&f.has_content_in_original_bundle=true,equals"

# if script fails start at a given page by changing page parameter here 
# url = "https://publica-rest.fraunhofer.de/server/api/discover/search/objects?configuration=defaultConfiguration&f.has_content_in_original_bundle=true,equals&page=379"

r = requests.request("GET", url) 
data = r.text
data = json.loads(data)

anzahl = data["_embedded"]["searchResult"]["page"]["totalElements"]
anzahl = int(anzahl)

pages = data["_embedded"]["searchResult"]["page"]["totalPages"]
pages = int(pages)

print("Total elements:  ", anzahl)
print("Number of pages: ", pages)


Total elements:   27938
Number of pages:  1397


In [72]:
# Create a function that extracts the relevant data from the request url

def get_data(): 
    global no_of_records
    for j in range(len(object)):
        # fetch some date from result page
        item_uuid = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"]
        item_handle = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["handle"]
          
        
        # create URL for more information about the item (api/core/items path)
        bundles_url = base_url_item + item_uuid + app_bundles
        bundles_r = requests.request("GET",bundles_url)
        bundles_data = bundles_r.text
        bundles_data = json.loads(bundles_data)
        file_exists = bundles_data["page"]["totalElements"] # if totalElements == 2 than two bitstream files are attached to this item
        print("Parse record: ", j, "with UUID: ", item_uuid, "has bitstream elements:", str(file_exists))
        
        if file_exists > 0:
            for k in range(file_exists):
                bundles_uuid = bundles_data["_embedded"]["bundles"][k]["uuid"]
                bundles_name = bundles_data["_embedded"]["bundles"][k]["name"]
                bundles_link = bundles_data["_embedded"]["bundles"][k]["_links"]["bitstreams"]["href"]
                
                bitstream_r = requests.request("GET", bundles_link)
                bitstream_data = bitstream_r.text
                bitstream_data = json.loads(bitstream_data)
                
                try: 
                    bitstream_uuid = bitstream_data["_embedded"]["bitstreams"][0]["uuid"]
                except: 
                    bitstream_uuid = "no bitstream UUId available"
                    bitstream_name = ""
                    bitstream_bundleName = ""
                    bitstream_size = ""
                    dwl_url = ""
                    orig_file_name = ""
                    file_format = ""
                else:
                    bitstream_uuid = bitstream_data["_embedded"]["bitstreams"][0]["uuid"]                
                    bitstream_name = bitstream_data["_embedded"]["bitstreams"][0]["name"]
                    file_format = bitstream_name.rsplit(".",1)
                    file_format = file_format[-1]
                    bitstream_bundleName = bitstream_data["_embedded"]["bitstreams"][0]["bundleName"]
                    bitstream_size = bitstream_data["_embedded"]["bitstreams"][0]["sizeBytes"]
                    dwl_url = base_url_download + bitstream_uuid + app_download
                    orig_file_name = ""
                try:
                    orig_file_name = bitstream_data["_embedded"]["bitstreams"][0]["metadata"]["dc.source"][0]["value"]
                except:
                    orig_file_name = "none"
                    orig_file_format = ""
                else: 
                    orig_file_name = bitstream_data["_embedded"]["bitstreams"][0]["metadata"]["dc.source"][0]["value"]
                    orig_file_format = orig_file_name.rsplit(".",1)
                    orig_file_format = orig_file_format[-1]
                
                mylist = [[item_uuid, item_handle, bundles_uuid, bundles_name, bitstream_uuid, bitstream_name, file_format, orig_file_name, orig_file_format, bitstream_bundleName, bitstream_size, dwl_url]]
                
                with open(output_file, 'a', newline='', encoding="utf-8") as f:
                    write = csv.writer(f)
                    write.writerows(mylist)        

In [73]:
# Url ausführen

r = requests.request("GET", url)
data = r.text
data = json.loads(data)

object = data["_embedded"]["searchResult"]["_embedded"]["objects"]
print("Parse page: 0")
get_data() # fetch data from the first page

if pages > 1: # if the response contains more than one page than iter trough pages
        for l in range(pages-1):
            url = data["_embedded"]["searchResult"]["_links"]["next"]["href"]
            
            r = requests.request("GET", url)
            data = r.text
            data = json.loads(data)
            object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

            print("Parse page:", (l + 1))
            get_data()
            
print("\n all things done.")

Parse page: 0
Parse record:  0 with UUID:  3ac2f74a-fa2e-4b78-b159-66ace57828b7 has bitstream elements: 2
Parse record:  1 with UUID:  0ea43579-5bb6-4120-8de6-93f98de83573 has bitstream elements: 2
Parse record:  2 with UUID:  1930c37c-7e40-4630-bcb2-98417c5add8d has bitstream elements: 2
Parse record:  3 with UUID:  93166ad9-c240-4b49-9437-2f81cf5697ef has bitstream elements: 2
Parse record:  4 with UUID:  8b0b8d2a-966c-419c-9751-b7ff64adc39d has bitstream elements: 2
Parse record:  5 with UUID:  c40e096a-012c-4a1e-a969-0af3c078efde has bitstream elements: 2
Parse record:  6 with UUID:  b3c12a0e-b0dd-4a29-b18d-2f06dc5a7817 has bitstream elements: 2
Parse record:  7 with UUID:  624621e5-d4f6-4fb8-8988-45958f25b058 has bitstream elements: 2
Parse record:  8 with UUID:  3e87ef3e-337e-4565-af11-6d1bc43fdb96 has bitstream elements: 2
Parse record:  9 with UUID:  5cc31257-78c3-4930-92f5-d32a25e94472 has bitstream elements: 2
Parse record:  10 with UUID:  a634b984-cf02-4644-9d44-872284efb7d6

Parse record:  8 with UUID:  52ffc8ee-f5fb-44cf-b33e-67fb9a87ac3b has bitstream elements: 2
Parse record:  9 with UUID:  62b70178-f530-4283-9cdd-ab139c0ab7ee has bitstream elements: 2
Parse record:  10 with UUID:  5f2c7ebb-55e2-47ea-8aba-1ec0eaf05d2c has bitstream elements: 2
Parse record:  11 with UUID:  58bfbd7f-2f86-4b73-bac5-b44f04d08a63 has bitstream elements: 2
Parse record:  12 with UUID:  529f38de-7285-40ea-82b0-acd9123f67b9 has bitstream elements: 2
Parse record:  13 with UUID:  5b13ff4a-aeab-4822-af39-95faca228b16 has bitstream elements: 2
Parse record:  14 with UUID:  1c52238d-a15a-430b-bd8b-247d466deb31 has bitstream elements: 2
Parse record:  15 with UUID:  101d924f-3a28-4f57-850f-9a3299c2ced1 has bitstream elements: 2
Parse record:  16 with UUID:  17a39daa-4dd4-4fce-962e-d2d06068b1d6 has bitstream elements: 2
Parse record:  17 with UUID:  17dbbec9-ccc1-4918-936d-006dff205153 has bitstream elements: 2
Parse record:  18 with UUID:  133feeb7-d27d-44cf-8a8f-3aaec7edcefa has b

Parse record:  16 with UUID:  b367c127-6ee1-47cd-9df5-d3e8efcf39e7 has bitstream elements: 2
Parse record:  17 with UUID:  b3b0e69e-f94a-4252-b9ea-cfb9823221d9 has bitstream elements: 2
Parse record:  18 with UUID:  abd3fb55-d04e-44c5-867d-f2a5f9af1964 has bitstream elements: 2
Parse record:  19 with UUID:  b52bb260-7f76-4f60-b784-436343774783 has bitstream elements: 2
Parse page: 9
Parse record:  0 with UUID:  b5ffa283-4574-4dd4-88fb-87544f455833 has bitstream elements: 2
Parse record:  1 with UUID:  b603722a-6820-4c32-86de-15d4081ddfc4 has bitstream elements: 2
Parse record:  2 with UUID:  b577f8b5-1417-4904-a582-e4afc7796bb8 has bitstream elements: 2
Parse record:  3 with UUID:  bbacb9a1-4329-4965-88e2-de872d9057a0 has bitstream elements: 2
Parse record:  4 with UUID:  b0450b6c-abaf-468d-9d70-bc48a54a562d has bitstream elements: 2
Parse record:  5 with UUID:  bd460816-bd47-4337-bee8-a6c67525a5bb has bitstream elements: 2
Parse record:  6 with UUID:  c60b21c7-f5b6-49fc-8572-5fb82700e

Parse record:  4 with UUID:  fed26cab-f5d6-450e-ba7f-dcfe0cd192b4 has bitstream elements: 2
Parse record:  5 with UUID:  fcead37e-49c8-4aae-899f-df41d53201f5 has bitstream elements: 2
Parse record:  6 with UUID:  fe86505a-4693-467a-a2ad-5e525434afc9 has bitstream elements: 2
Parse record:  7 with UUID:  fbaefb3b-f6f6-4600-9c8c-49406ec869ba has bitstream elements: 2
Parse record:  8 with UUID:  fc847672-f406-4ae1-ab10-018c82bdc379 has bitstream elements: 2
Parse record:  9 with UUID:  a126bc60-8e1d-4b04-a483-5953e2b12ac7 has bitstream elements: 2
Parse record:  10 with UUID:  c0b3fd23-cf2a-4171-9b40-a5797f598d65 has bitstream elements: 2
Parse record:  11 with UUID:  9ef06ec3-ebec-4ca6-83e8-8a537a2ffcc3 has bitstream elements: 2
Parse record:  12 with UUID:  6f7ada26-d6c9-4d31-a6dd-594d5ffd9179 has bitstream elements: 2
Parse record:  13 with UUID:  e4dba962-b12f-429a-bdbc-729ab8f2d6a8 has bitstream elements: 2
Parse record:  14 with UUID:  3854a37c-9c8b-4648-9008-631e9dd1ea5c has bitst

Parse record:  12 with UUID:  1527b946-6b77-4528-a6c9-c026edd528bf has bitstream elements: 2
Parse record:  13 with UUID:  26f97b26-92f2-4c10-bfab-035b6db3ffc0 has bitstream elements: 2
Parse record:  14 with UUID:  b202cd89-48ee-49d8-815f-3c8c98e8d54e has bitstream elements: 2
Parse record:  15 with UUID:  6db38aaa-c01f-4e1a-afe8-8a9eabd1e626 has bitstream elements: 2
Parse record:  16 with UUID:  ec4488bf-e236-42df-84b7-abda57f36a87 has bitstream elements: 2
Parse record:  17 with UUID:  4e77da67-17c9-4f09-a6b2-dda6a8d35f32 has bitstream elements: 2
Parse record:  18 with UUID:  20924ebf-6218-4af5-9547-013532d67008 has bitstream elements: 2
Parse record:  19 with UUID:  7caab23b-fe17-4bdf-abf0-3e519b90d1c0 has bitstream elements: 2
Parse page: 18
Parse record:  0 with UUID:  74f27951-99ac-4edf-9cae-479e020cd2ff has bitstream elements: 2
Parse record:  1 with UUID:  77ef6360-a0d5-4b9d-92d4-0545b228f279 has bitstream elements: 2
Parse record:  2 with UUID:  a69ac151-4318-423b-85e6-1bc3

...

Parse record:  4 with UUID:  1e4ba82b-dd64-4f07-9861-3a7559553b3d has bitstream elements: 2
Parse record:  5 with UUID:  23b38189-b5fc-4583-805f-6d75f81f15dc has bitstream elements: 2
Parse record:  6 with UUID:  278100f5-0735-4347-bcab-f57e52adca9f has bitstream elements: 2
Parse record:  7 with UUID:  26e8fc9e-c6af-44f9-9dcb-9d99ea795efd has bitstream elements: 2
Parse record:  8 with UUID:  35f5e482-ab5b-4d97-831a-82665831823b has bitstream elements: 2
Parse record:  9 with UUID:  0b95c30f-324a-4921-bfdb-77174b43b442 has bitstream elements: 2
Parse record:  10 with UUID:  0c4f95a4-2d9a-4784-b4e7-7beac66b8996 has bitstream elements: 2
Parse record:  11 with UUID:  0d547115-39ed-4442-b9d3-7b1f955fe5fd has bitstream elements: 2
Parse record:  12 with UUID:  0143caff-44b4-4a9e-b118-fa30bb6bdb61 has bitstream elements: 2
Parse record:  13 with UUID:  8c4c9385-1a02-40bf-9d7e-ece9ff35dec0 has bitstream elements: 2
Parse record:  14 with UUID:  861f3a26-2519-4d7a-8f87-540023f6e778 has bitst

Parse record:  12 with UUID:  5d0fcf87-f287-4324-99b3-20b1d4cf0c65 has bitstream elements: 2
Parse record:  13 with UUID:  5c1d6586-bcb8-42ab-8c24-72bc985a37b0 has bitstream elements: 2
Parse record:  14 with UUID:  5c9fddc7-f154-4252-8499-97383c3be19b has bitstream elements: 2
Parse record:  15 with UUID:  1d4d7db8-1802-4b98-a33f-da82688f8e75 has bitstream elements: 1
Parse record:  16 with UUID:  12d97775-bada-446d-8cfe-e0b0df1162dd has bitstream elements: 2
Parse record:  17 with UUID:  26a2ffa0-2c40-439b-9d2b-4f8982c88c36 has bitstream elements: 2
Parse record:  18 with UUID:  351305fe-7a80-4807-a0dc-e1fc2b24d0eb has bitstream elements: 2
Parse record:  19 with UUID:  058bdbde-1fe3-4244-8ce5-8e80b09d2908 has bitstream elements: 1
Parse page: 1382
Parse record:  0 with UUID:  090d6bc2-f611-4487-8a73-39cc6ae9485a has bitstream elements: 2
Parse record:  1 with UUID:  0a544ee2-73e9-4eb6-a2f1-e1351f1370fe has bitstream elements: 2
Parse record:  2 with UUID:  0104614b-e94f-4145-8b8b-1d

Parse page: 1386
Parse record:  0 with UUID:  1067b413-c37a-4ae2-9536-bede05b438ea has bitstream elements: 2
Parse record:  1 with UUID:  86fe51cd-9b7c-437a-b62c-d13af721a4ae has bitstream elements: 2
Parse record:  2 with UUID:  b8f2cef3-c6b1-4fb7-aac5-5eb0ca8a2d5f has bitstream elements: 1
Parse record:  3 with UUID:  abb09713-4d67-4df6-a745-4080aa8d509b has bitstream elements: 2
Parse record:  4 with UUID:  bd2d2249-3f15-4a2c-ba56-2b84e30313e8 has bitstream elements: 2
Parse record:  5 with UUID:  d9d49e91-6e4a-4307-ac02-602a3d8a4acd has bitstream elements: 2
Parse record:  6 with UUID:  3b34a1f1-d257-430c-8f9c-e2d4dd02e944 has bitstream elements: 2
Parse record:  7 with UUID:  3e55edaa-814c-400a-89aa-6dd9df1614de has bitstream elements: 2
Parse record:  8 with UUID:  412a2d24-de7d-4421-ad4e-194e3d53367e has bitstream elements: 2
Parse record:  9 with UUID:  3de575dc-ab08-4d3a-abbb-a2e717a4413e has bitstream elements: 2
Parse record:  10 with UUID:  60cba8dc-3019-4eb5-8598-b901606ca

Parse record:  8 with UUID:  411616e5-ac25-4316-ac66-dc9ded7a12be has bitstream elements: 2
Parse record:  9 with UUID:  1c8cf4fe-afaf-447e-bd76-d491eae27051 has bitstream elements: 2
Parse record:  10 with UUID:  32c5734a-1e22-4d1d-a337-d3563c373b3e has bitstream elements: 2
Parse record:  11 with UUID:  32a073eb-4e95-4785-9707-c6e34c08fc5f has bitstream elements: 2
Parse record:  12 with UUID:  7e3903fb-3e0e-457d-a833-c12da6d512eb has bitstream elements: 2
Parse record:  13 with UUID:  7b5ad436-ccf3-4365-8fb4-5a777ec5329f has bitstream elements: 2
Parse record:  14 with UUID:  b36cf30f-dc8a-4414-ab12-a63c82eef5ed has bitstream elements: 2
Parse record:  15 with UUID:  ba875c1f-b4e9-4d99-9392-472fe0dad00b has bitstream elements: 2
Parse record:  16 with UUID:  c8ca1c5f-e419-4255-bf4f-133dcf2b12ce has bitstream elements: 2
Parse record:  17 with UUID:  d52660d7-bdce-423f-a3b8-b0362f0bf271 has bitstream elements: 2
Parse record:  18 with UUID:  9ac8c58d-2de5-4ef9-86ad-436ed13a3185 has b

Parse record:  16 with UUID:  bce2404a-4a3b-48b3-ad3f-7a8187cc8e23 has bitstream elements: 1
Parse record:  17 with UUID:  bce58d25-0457-45d2-a86c-a171d1f2973d has bitstream elements: 1
Parse record:  18 with UUID:  d1e520a5-99b4-411d-a3c2-85ba59f2cff6 has bitstream elements: 1
Parse record:  19 with UUID:  ca766e96-ceda-4091-9bea-4a630f83e9f2 has bitstream elements: 1
Parse page: 1395
Parse record:  0 with UUID:  cf0c11af-86d6-4eec-9dcd-74beade1a3c3 has bitstream elements: 1
Parse record:  1 with UUID:  c129805d-5693-440f-9e8e-31b03e400b10 has bitstream elements: 1
Parse record:  2 with UUID:  9789a91d-5031-4104-963a-8a394a97c23f has bitstream elements: 1
Parse record:  3 with UUID:  9a01651c-26c2-4fd5-b737-b4864f935ec6 has bitstream elements: 1
Parse record:  4 with UUID:  9c7085b8-a77b-4f56-9efc-75caaf511097 has bitstream elements: 1
Parse record:  5 with UUID:  a1142154-fd48-4f43-b81a-d894bbab0991 has bitstream elements: 1
Parse record:  6 with UUID:  9e88ae46-ec75-42d0-af0a-6c0542